In [6]:
import os
from deepface import DeepFace
import cv2
from tqdm import tqdm
import numpy as np
import time

2025-01-28 13:48:09.481039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738072089.497346 1480992 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738072089.502594 1480992 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 13:48:09.523767: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model_name = "Facenet512"
detector_backend = "mtcnn"

In [8]:
embeddings = []
file_path = []

for dirpath, dirnames, filenames in os.walk("/home/playground/candidate_verification/images/Humans"):
   for file in tqdm(filenames[:20]):
      try:  
            img_path = f"{dirpath}/{file}"
            result =  DeepFace.represent(img_path = img_path, model_name="Facenet512", detector_backend="mtcnn")            
            for r in result:
                  if r: 
                     embeddings.append(r.get("embedding"))
                     file_path.append(img_path)      
      
      except Exception as e:
         print(e)
               



 45%|████▌     | 9/20 [00:15<00:20,  1.85s/it]

Face could not be detected in /home/playground/candidate_verification/images/Humans/1 (2011).jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


100%|██████████| 20/20 [00:44<00:00,  2.23s/it]


#### Qdrant

In [14]:
from ast import List
import nest_asyncio
nest_asyncio.apply()


from qdrant_client import AsyncQdrantClient, models
from qdrant_client.models import Distance, VectorParams, PointStruct

client = AsyncQdrantClient(url="http://localhost/",port=6333)

async def create_collection (collection_name):
    try:
        await client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=512, distance=Distance.EUCLID),
        optimizers_config=models.OptimizersConfigDiff(memmap_threshold=20000),
        hnsw_config=models.HnswConfigDiff(on_disk=True)
        
        )
         
    except Exception as e:
        print(e)

async def deletect_collection(collection_name):
        try:
            result = await client.delete_collection(collection_name=f"{collection_name}")
            return result
        except Exception as e:
            print(e)

async def get_collection_info(collection_name):
        try:
            result = await client.get_collection(collection_name)
            return result    
        except Exception as e:
            print(e)

async def add_point_to_collection(collection_name, point_data):
        try:
            operation_info = await client.upsert(
            collection_name=collection_name,
            wait=True,
            points=models.Batch(
                 vectors=point_data.get('embedding'),
                 payloads=point_data.get('payload')
            ))

            return operation_info
        except Exception as e:
             print(e)       

In [3]:
res = await create_collection("face-recognition")

In [28]:
res = await deletect_collection("sample_collection")

In [10]:
res = await get_collection_info("face-recognition")
print(res)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=0 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=512, distance=<Distance.EUCLID: 'Euclid'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=True, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=20000, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None, 

In [15]:
point_data = {"embedding":embeddings,"payload":file_path}


In [22]:

embedding = np.array(point_data.get('embedding')[0])

(512,)

In [ ]:
point_data

In [ ]:
point_data

In [17]:
res = await add_point_to_collection("face-recognition", point_data)
print(res)

20 validation errors for Batch
ids
  Field required [type=missing, input_value={'vectors': [[0.194046944...es/Humans/1 (597).jpg']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
payloads.0
  Input should be a valid dictionary [type=dict_type, input_value='/home/playground/candida...ges/Humans/1 (4985).jpg', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
payloads.1
  Input should be a valid dictionary [type=dict_type, input_value='/home/playground/candida...ges/Humans/1 (4707).jpg', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
payloads.2
  Input should be a valid dictionary [type=dict_type, input_value='/home/playground/candida...ges/Humans/1 (6894).jpg', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
payloads.3
  Input should be a valid dictionary [type=dict_type, input_value='/home/playground/